In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from Likelihood import Likelihood
from functools import partial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of fun5 failed: Traceback (most recent call last):
  File "/Users/emmaknippel/anaconda3/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/emmaknippel/anaconda3/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Users/emmaknippel/anaconda3/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun5.py", line 3, in <module>
    from fun0 import pow
ModuleNotFoundError: No module named 'fun0'
]


In [12]:
from fun0 import pow


ModuleNotFoundError: No module named 'fun0'

In [6]:
def load_csv(filename, folder):
    return pd.read_csv(os.path.join(folder, f"{filename}.csv"), header=0).values

folder = "/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Replicating the Innovators Dilemma/1 Summary Statistics/Data"
State = load_csv("State", folder)
Adopt = load_csv("Adopt", folder)
Exit = load_csv("Exit", folder)

Pi = loadmat('CheckPi.mat')['Pi']

In [3]:
T = Pi.shape[0]
beta = 0.88      # Discount factor
delta = 0.9388   # Decline in SC
scaling = 1000   # Scaling factor
iterMLE = 1

print(f"  -----------------------------------------")
print(f" | STRATEGIC (RATIONAL-EXPECTATIONS) MODEL |")
print(f"  -----------------------------------------")

print(f" ----- Basic parameters to be \"calibrated\" -----")
print(f"   Profit/value scaling      :     {scaling:4.0f}")
print(f"   Discount factor (beta)    :     {beta:1.4f}")
print(f"   Adoption cost drop (delta):     {delta:1.4f}")
print(" -----------------------------------------------\n")

# Rescale profits
Pi = Pi / scaling

# Terminal Values
V = np.zeros((T, 3, 12, 12, 15))
V[T-1,:,:,:,:] = Pi[T-1,:,:,:,:] / (1 - beta)

# Initialize Solutions
EV = np.zeros((T, 5, 12, 12, 15))
Policy = np.zeros((T-1, 5, 12, 12, 15))

# MLE Estimation
x0 = [1, 1, 1]
options = {'disp': True, 'maxiter': 1000, 'xatol': 1e-4, 'fatol': 1e-8}

objective_function = partial(Likelihood, beta=beta, delta=delta, Pi=Pi, V=V, EV=EV, Policy=Policy, State=State, Exit=Exit, Adopt=Adopt, T=T, iterMLE=iterMLE, output_type=1)

res = minimize(objective_function, x0, method='Nelder-Mead', options=options)
Theta = res.x

# Numerical derivatives for standard errors
perturb = 0.001
logL = Likelihood(Theta, 2)
logLderiv = np.zeros((1, 3))

for k in range(3):
    Theta_perturbed = np.copy(Theta)
    Theta_perturbed[k] += perturb
    logL_perturbed = Likelihood(Theta_perturbed, 2)
    logLderiv[0, k] = (logL_perturbed - logL) / perturb

# Derive the Value estimate
Vhat = np.linalg.inv(logLderiv.T @ logLderiv)

print(f"\n ----- Maximum Likelihood Estimation Results for (phi & kappa_inc) -----")
print(f" Coeff.: " + " ".join([f"{v:8.8f}" for v in Theta]))
print(f" S.E.  : " + " ".join([f"{v:8.8f}" for v in np.sqrt(np.diag(Vhat))]))
print(f" -----------------------------------------------------------------------\n")

  -----------------------------------------
 | STRATEGIC (RATIONAL-EXPECTATIONS) MODEL |
  -----------------------------------------
 ----- Basic parameters to be "calibrated" -----
   Profit/value scaling      :     1000
   Discount factor (beta)    :     0.8800
   Adoption cost drop (delta):     0.9388
 -----------------------------------------------


MLE iter #   1: trying (phi, kappa_inc, kappa_ent) = (1.00000000, 1.00000000, 1.00000000)
Year: 16, State    1: (no, nb, nn) = ( 0,  0,  0)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 0.00, 0.00, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, 0.0000, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 0.00, 0.00)
Year: 16, State    2: (no, nb, nn) = ( 0,  0,  1)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 0.00, 0.00, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, 0.2605, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 0.00, 9.04)
Year: 16, State    3: (no, nb, nn) = ( 0,  0,  2)
E

/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun9.py:17: RuntimeWarning: overflow encountered in exp
  stay = np.exp(phi + beta * z4)
/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun13.py:16: RuntimeWarning: overflow encountered in exp
  stay = np.exp(phi + beta * z4)


Year: 14, State   52: (no, nb, nn) = ( 0,  3,  6)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 62.03, 70.63, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, 0.0000, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 59.30, 65.56)
Year: 14, State   53: (no, nb, nn) = ( 0,  3,  7)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 43.72, 51.02, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, 0.0000, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 43.01, 48.10)
Year: 14, State   54: (no, nb, nn) = ( 0,  3,  8)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 46.04, 45.17, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, 0.0000, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 44.84, 42.75)
Year: 14, State   55: (no, nb, nn) = ( 0,  3,  9)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 19.02, 18.66, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, 0.0000, 0.0000)
Value today (Vo, Vb, Vn) = (

/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun6.py:17: RuntimeWarning: overflow encountered in exp
  stay = np.exp(phi + beta * z1)
/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun7.py:19: RuntimeWarning: overflow encountered in exp
  stay = np.exp(phi + beta * z1)
/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun11.py:18: RuntimeWarning: overflow encountered in exp
  stay = np.exp(phi + beta * z1)


Year: 14, State  358: (no, nb, nn) = ( 1, 11, 12)
Expected Value (z1, z2, z3, z4, z5) = (17.96, 0.00, 19.04, 12.87, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, 0.0000, 0.0000)
Value today (Vo, Vb, Vn) = (17.51, 18.89, 13.33)
Year: 14, State  359: (no, nb, nn) = ( 1, 11, 13)
Expected Value (z1, z2, z3, z4, z5) = (15.26, 0.00, 15.87, 12.98, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, 0.0000, 0.0000)
Value today (Vo, Vb, Vn) = (15.15, 16.06, 13.37)
Year: 14, State  360: (no, nb, nn) = ( 1, 11, 14)
Expected Value (z1, z2, z3, z4, z5) = (2.03, 0.00, 15.72, 1.94, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0483, 0.0846, 0.0000, 0.0608, 0.0000)
Value today (Vo, Vb, Vn) = (3.67, 15.88, 3.67)
Year: 14, State  361: (no, nb, nn) = ( 2,  0,  0)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 2.80, 0.00, 0.00, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0368, 0.8474, 0.0000, 0.0000, 0.0000)
Value today (Vo, Vb, Vn) = (

/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun4.py:120: RuntimeWarning: invalid value encountered in double_scalars
  EV4 += BS4[no_prime + 11*nb_prime + (11*11)*nn_prime + (11*11*14)*npe_prime] \


Year: 13, State   57: (no, nb, nn) = ( 0,  3, 11)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 56.84, nan, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, nan, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 51.73, nan)
Year: 13, State   58: (no, nb, nn) = ( 0,  3, 12)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 103.29, nan, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, nan, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 92.68, nan)
Year: 13, State   59: (no, nb, nn) = ( 0,  3, 13)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 82.27, nan, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, nan, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 74.09, nan)
Year: 13, State   60: (no, nb, nn) = ( 0,  3, 14)
Expected Value (z1, z2, z3, z4, z5) = (0.00, 0.00, 64.20, nan, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (0.0000, 0.0000, 0.0000, nan, 0.0000)
Value today (Vo, Vb, Vn) = (0.00, 58.17, nan)
Year: 1

/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Term Paper/ReplicatingInnovatorsDilemma/3 Maximum Likelihood/fun1.py:110: RuntimeWarning: invalid value encountered in double_scalars
  EV1 += BS1[no_prime + 11*nb_prime + (11*11)*nn_prime + (11*11*14)*npe_prime] \


Year: 13, State  202: (no, nb, nn) = ( 1,  1,  6)
Expected Value (z1, z2, z3, z4, z5) = (nan, 0.00, nan, nan, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (nan, nan, nan, nan, 0.0000)
Value today (Vo, Vb, Vn) = (nan, nan, nan)
Year: 13, State  203: (no, nb, nn) = ( 1,  1,  7)
Expected Value (z1, z2, z3, z4, z5) = (nan, 0.00, nan, nan, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (nan, nan, nan, nan, 0.0000)
Value today (Vo, Vb, Vn) = (nan, nan, nan)
Year: 13, State  204: (no, nb, nn) = ( 1,  1,  8)
Expected Value (z1, z2, z3, z4, z5) = (nan, 0.00, nan, nan, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (nan, nan, nan, nan, 0.0000)
Value today (Vo, Vb, Vn) = (nan, nan, nan)
Year: 13, State  205: (no, nb, nn) = ( 1,  1,  9)
Expected Value (z1, z2, z3, z4, z5) = (nan, 0.00, nan, nan, 0.00)
Choice Probabilities (z6, z7, z8, z9, z10) = (nan, nan, nan, nan, 0.0000)
Value today (Vo, Vb, Vn) = (nan, nan, nan)
Year: 13, State  206: (no, nb, nn) = ( 1,  1, 10)
Expected Value (z1

Warning: Error fun1: Vprime must have 6480 elements